In [7]:
import pandas as pd
import spacy
dataframe = pd.read_csv('training_set_rel3.tsv', encoding = 'latin-1',sep='\t')
dataframe = dataframe[['essay_id','essay_set','essay','domain1_score']]
nlp = spacy.load('en_core_web_md')
essays = dataframe
# ----------- Isolate essays from the 6th set ------------ #
essays = essays[(essays['essay_set'] == 6)]
essays.dropna(axis=1, how='all', inplace=True)
maxdf = essays[essays.domain1_score == essays.domain1_score.max()].iloc[0]
maxdf

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


essay_id                                                     14836
essay_set                                                        6
essay            The builders of the Empire State Building face...
domain1_score                                                    4
Name: 8886, dtype: object

In [8]:
maxdf.essay

"The builders of the Empire State Building faced many obstacles while attempting to allow dirigibles to dock there. one obstacle was that the dirigible would have to be held by a single cable tether. This would add stress to the building's frame in order to make the frame or sturdy, there was modifications to the building's framework which cost over sixty thousand dollars. Another obstacle the builders faced was the dirigible's use of hydrogen which is highly flammable. Since New York was a densely populated area, this would not work out well. The dirigibles would also have to be weighted down in the back with lead weights. In order to do so, the lead weights would end up dangling high above pedestrians on the street which was a safety issue one last obstacle the builders faced was the law against airships flying too low over urban areas. This law made it illegal for a ship to ever tie up to the building or approach the area."

In [9]:
from rake_nltk import Rake
r = Rake(max_length=1) # Uses stopwords for english from NLTK, and all puntuation characters.
r.extract_keywords_from_text(maxdf.essay)
ini = r.get_ranked_phrases()


In [12]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
fin = set()
for word in ini:
    syns = wordnet.synsets(lemmatizer.lemmatize(word))
    if len(syns) != 0:
        syn = syns[0]
        lemmas = syn.lemmas()
        for lemma in lemmas:
            fin.add(lemma.name())
fin

{'H',
 'adjustment',
 'airship',
 'alteration',
 'approach',
 'area',
 'assay',
 'atomic_number_1',
 'attack',
 'attempt',
 'back',
 'brand',
 'builder',
 'building',
 'burden',
 'burthen',
 'cost',
 'country',
 'depression',
 'detergent_builder',
 'dirigible',
 'dock',
 'dorsum',
 'edifice',
 'employment',
 'essay',
 'exercise',
 'footer',
 'frame',
 'framework',
 'hardy',
 'hold',
 'hydrogen',
 'illegal',
 'jurisprudence',
 'keep',
 'law',
 'low',
 'maintain',
 'make',
 'model',
 'modification',
 'order',
 'pedestrian',
 'plan_of_attack',
 'seek',
 'ship',
 'stalwart',
 'stout',
 'street',
 'sturdy',
 'theoretical_account',
 'try',
 'usage',
 'use',
 'utilisation',
 'utilization',
 'walker',
 'weight',
 'weight_down',
 'well',
 'work'}

In [14]:
def wordnettopic(df):
    col = []
    for essay in df['essay']:
        r.extract_keywords_from_text(essay)
        ini = r.get_ranked_phrases()
        tot = 0
        for word in ini:
            if lemmatizer.lemmatize(word) in fin:
                tot+=1
        col.append(tot)
    return df.assign(wordnetscore = col)
essays = wordnettopic(essays)
essays.head()

,essay_id,essay_set,essay,domain1_score,wordnetscore
8884,14834,6,There were many obstacles that the builders fa...,2,3
8885,14835,6,"Him from the start, there would have been many...",3,6
8886,14836,6,The builders of the Empire State Building face...,4,24
8887,14837,6,In the passage The Mooring Mast by Marcia Amid...,1,4
8888,14838,6,The builders of the Empire State Building face...,3,8


In [15]:
essays['domain1_score'].corr(essays['wordnetscore'])

0.6292573944197947